In [2]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import votable, ascii
# import h5py
import json
import numpy as np
from types import SimpleNamespace
from astropy.table import Table, vstack

### SQL query on MAST

In [3]:
def z_cat_query(tl, br):
    string = "select * from catalogRecordRowStore where raMean>{:.2f} and raMean<{:.2f} and decMean>{:.2f} and decMean<{:.2f}".format(br[0], tl[0], br[1], tl[1])
    return string

In [4]:
def sexa_to_deci(c):
    sc = SkyCoord(c, unit=(u.hourangle, u.deg))
    return sc.ra.deg, sc.dec.deg

In [5]:
br = sexa_to_deci("00:03:19.1890576 +09:54:13.899849")
tl = sexa_to_deci("00:04:35.3301991 +10:12:15.172584")

In [6]:
print("test<{:.2f}".format(br[0]))

test<0.83


In [7]:
print(z_cat_query(tl, br))

select * from catalogRecordRowStore where raMean>0.83 and raMean<1.15 and decMean>9.90 and decMean<10.20


### Preparing prioritize.py input

In [8]:
center = "00:03:48.8888534 +10:04:10.737999"
print(sexa_to_deci(center))
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
sc.ra.hour

(0.9537035558333332, 10.069649444166666)


0.06358023705555556

In [9]:
center = "00:03:50.0443707 +10:04:22.110113"
print(sexa_to_deci(center))
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
sc.ra.hour

(0.9585182112499999, 10.072808364722222)


0.06390121408333334

The following cells are just a test.

In [18]:
archivalz = ascii.read('archivalz.csv')
list(archivalz['RA'].data)

In [20]:
v = SimpleNamespace()

In [26]:
io_dir = '1/'
align = "{}align.xml".format(io_dir)
try:
    votb2 = (votable.parse_single_table(source=align)).to_table()
    v.aligninfo = list(votb2.iterrows('_RAJ2000','_DEJ2000','rmag'))
except:
    raise SystemExit('No align file.')

In [22]:
votb2.columns

<TableColumns names=('_RAJ2000','_DEJ2000','_V','mode','q_mode','cl','SDSS8','m_SDSS8','Im','RA_ICRS','DE_ICRS','zsp','umag','e_umag','gmag','e_gmag','rmag','e_rmag','imag','e_imag','zmag','e_zmag','Q','ObsDate','objID')>

In [23]:
print(list(votb2.iterrows('_RAJ2000','_DEJ2000','rmag')))

[(0.948284, 9.962229, 15.765), (1.001962, 9.969801, 19.387), (0.875299, 10.185863, 15.263), (0.846498, 10.140442, 15.871), (0.859229, 10.16519, 18.891)]


In [24]:
v.aligninfo

[(0.948284, 9.962229, 15.765),
 (1.001962, 9.969801, 19.387),
 (0.875299, 10.185863, 15.263),
 (0.846498, 10.140442, 15.871),
 (0.859229, 10.16519, 18.891)]

### Converting archival redshift file into csv

In [69]:
archival = ascii.read('archivalz.txt', names = ['RA', 'DEC', 'z', 'delta_z'])
archival.write('archivalz.csv', format='ascii.csv')

### Downloading the PS catalog

In [12]:
import ps_api

In [15]:
# cl_pos = ps_api.resolve('RMJ000343.8+100123.8')
cl_pos = sexa_to_deci(center)
cl_pos

(0.9537035558333332, 10.069649444166666)

In [16]:
# radius = (9 * u.arcmin).to(u.deg).value
radius = (12 * u.arcmin).to(u.deg).value
# radius = .03

In [25]:
# constraints = {'nDetections.gt':1, 'rMeanKronMag.gt':-990}
# constraints = {'rMeanKronMag.gt':-990}
constraints = {'nDetections.gt':10}
# constraints = {}

In [26]:
columns = """objID,raMean,decMean,rMeanKronMag,nDetections""".split(',')
# strip blanks and weed out blank and commented-out values
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

In [27]:
results = ps_api.ps1cone(*cl_pos, radius, columns=columns, **constraints)
# try including release="dr2" as an argument

In [28]:
with open('ps_api_res_nDetections10.csv', 'w') as f:
    f.write(results)

### Converting output.txt into catalog

In [22]:
# io_dir = '1/dsim/'
io_dir = '2/dsim/'
columns = ['OBJNAME', 'RA', 'DEC', 'EQX', 'MAG', 'band', 'PCODE', 'LIST', 'SEL?', 'PA', 'L1', 'L2']
dtype = []

for column in columns:
    if column=='SEL?':
        dtype.append('i4')
    else:
        dtype.append('<U12')

def read_target_file(filename):
    with open('{}{}'.format(io_dir, filename),'r') as targets:
        lines = targets.readlines()

    align = []
    targets = []
    for line in lines:
        line_data = line.strip().split()
        if len(line_data)==9:
            align.append(line_data)
        elif len(line_data)==10:
            targets.append(line_data)
    return align, targets


In [23]:
infilename = "output.txt".format(io_dir)
out_selected = "{}selected.csv".format(io_dir)
out_not_selected = "{}not_selected.csv".format(io_dir)
out_align = "{}align.csv".format(io_dir)

In [24]:
align, targets = read_target_file(infilename)

In [25]:
df_targets = Table(np.array(targets), names=columns[:-2], dtype=dtype[:-2])
df_align = Table(np.array(align[:-1]), names=columns[:-3], dtype=dtype[:-3])

In [26]:
df_targets[df_targets['SEL?']==1].write(out_selected, format='ascii.csv', overwrite=True)
df_targets[df_targets['SEL?']==0].write(out_not_selected, format='ascii.csv', overwrite=True)
df_align.write(out_align, format='ascii.csv', overwrite=True)

In [27]:
df_targets[df_targets['SEL?']==1]

OBJNAME,RA,DEC,EQX,MAG,band,PCODE,LIST,SEL?,PA
str12,str12,str12,str12,str12,str12,str12,str12,int32,str12
9045648334,00:03:37.158,+09:57:23.64,2000.0,21.00,r,637,1,1,-98
8541609913,00:03:25.014,+09:58:58.10,2000.0,20.61,r,4961,1,1,-98
9363621614,00:03:44.726,+09:58:33.22,2000.0,20.88,r,744,1,1,-98
9340009833,00:03:44.164,+09:58:27.83,2000.0,20.48,r,4232,1,1,-98
8937131009,00:03:34.515,+10:00:01.01,2000.0,20.73,r,1970,1,1,-98
9097560626,00:03:38.352,+09:59:30.18,2000.0,19.75,r,14642,1,1,-98
8880001157,00:03:33.128,+10:01:31.96,2000.0,20.63,r,2364,1,1,-98
9190586401,00:03:40.579,+10:01:47.58,2000.0,20.66,r,3906,1,1,-98
9202491693,00:03:40.861,+10:01:33.55,2000.0,20.07,r,5845,1,1,-98


### Stacking NED and SDSS catalogs
Not used. May contain duplicates.

In [61]:
david1 = ascii.read('archivalz_david1.csv')
ned = ascii.read('archivalz_ned.csv')
vstack([david1, ned])